In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from OptWind import windOpt_linear
from OptJoint import jointOpt_linear
from OptStorage import storageOpt_linear
from OptJoint import jointOpt_linear_noArb
from OptDisjoint import disjointOpt_linear
import matplotlib.pyplot as plt

In [2]:
res_profile = np.load('Interresult\wind_summer.npz')['data']
print(res_profile)

[41.23934783 39.70043478 37.61673913 35.61956522 33.69608696 31.64869565
 28.80315217 25.5251087  26.89293478 25.08630435 22.30728261 20.73630435
 21.01293478 22.53380435 24.60728261 26.87586957 29.15423913 31.61347826
 33.67141304 34.4425     36.89467391 40.35869565 42.52902174 42.46423913]


In [3]:
netload_profile = np.load('Interresult\loadnet_summer.npz')['data']
print(netload_profile)

[35.68603228 34.23154913 33.61755641 33.69469033 34.66817533 36.0471725
 36.57817337 36.64509239 37.74290826 41.5130763  46.0877262  50.11039989
 53.02994293 54.77400522 55.4589575  55.56166522 55.00097315 54.07419652
 53.92191859 53.71014609 50.83260348 45.9857975  41.52933576 37.93567674]


#### \alpha and \beta

In [4]:
T = len(netload_profile)
alpha = np.zeros(T)
beta_base = np.zeros(T)
for i, d in enumerate(netload_profile):
    if d>=10.9 and d<=51.0:
        alpha[i] = 25.94 + 0.12 *d
        beta_base[i] = 0.12 
    elif d> 51.0 and d<= 58.0:
        alpha[i] = -82.25 + 2.28 *d
        beta_base[i] = 2.28
    elif d>=58.0 and d<= 67.8:
        alpha[i] =  -0.47 + 0.8 *d
        beta_base[i] = 0.8
    else:
        print('ERROR!')
print(alpha)
print(beta_base)

[30.22232387 30.0477859  29.97410677 29.98336284 30.10018104 30.2656607
 30.3293808  30.33741109 30.46914899 30.92156916 31.47052714 31.95324799
 38.65826989 42.6347319  44.1964231  44.4305967  43.15221879 41.03916807
 40.69197438 40.20913308 32.03991242 31.4582957  30.92352029 30.49228121]
[0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 2.28 2.28
 2.28 2.28 2.28 2.28 2.28 2.28 0.12 0.12 0.12 0.12]


In [5]:
beta= np.diag(beta_base)

#### Exploring possibility

In [6]:
storage_choice = np.arange(0.2, 2, 0.2)
RES_choice = np.arange(0.2, 2, 0.2)
n_storage = len(storage_choice)
n_RES = len(RES_choice)

In [7]:
storage_choice = np.arange(0.2, 2, 0.2)
RES_choice = np.arange(0.2, 2, 0.2)
n_storage = len(storage_choice)
n_RES = len(RES_choice)

In [8]:
m_joint_record = np.zeros((n_storage, n_RES))
m_disjoint_record = np.zeros((n_storage, n_RES))
w_single_record = np.zeros((n_storage, n_RES))
m_joint_record_noArb = np.zeros((n_storage, n_RES))
valueWind_record = np.zeros((n_storage, n_RES)) 
valueWind_record_noArb = np.zeros((n_storage, n_RES)) 
value_arbitrager_record = np.zeros((n_storage, n_RES))

In [9]:
#### Example: storage 1GW, wind 1GW
for i, s_cap in enumerate(storage_choice):
        for j, w_cap in enumerate(RES_choice):
                s_ub = s_cap* np.ones(T)
                s_ub[-1] = s_ub[-1] - 0.5*s_cap
                s_lb = 0* np.ones(T)
                s_lb[-1] = s_lb[-1] + 0.5*s_cap
                s_init = np.zeros(T)
                s_init[0] = 0.5 * s_cap
                eta_c = 1
                eta_d = 1

                # w_cap = 1
                w_true = res_profile * w_cap /100

                w, x, s, m = jointOpt_linear(T, alpha, beta, w_true, s_ub, s_lb, s_init, eta_c, eta_d)
                w_n, x_n, s_n, m_n = jointOpt_linear_noArb(T, alpha, beta, w_true, s_ub, s_lb, s_init, eta_c, eta_d)
                w2_linear, m2_linear = windOpt_linear(T, alpha, beta, w_true)
                valueWind_record[i,j] = m-m2_linear
                valueWind_record_noArb[i,j] = m_n-m2_linear
                m_joint_record[i,j] = m
                w_single_record[i,j]= m2_linear
                m_joint_record_noArb[i,j] = m_n
                eps = 1e-3
                kUb = 10
                w_sol_try, x_sol_try, m_disjoint_wind_try, m_disjoint_storage_try=disjointOpt_linear(T, alpha, beta, w_true, s_ub, s_lb, s_init, eta_c, eta_d, eps, kUb)
                m_disjoint_record[i,j] = m_disjoint_wind_try+ m_disjoint_storage_try
                value_arbitrager_record[i,j] = m_disjoint_storage_try

# print("profit", m_joint_record, m_disjoint_record)
# print("Value", valueWind_record, valueWind_record_noArb, value_arbitrager_record)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-27


In [ ]:
np.savez('.\Interresult\heatmap_wind_summer.npz', 
         m_joint = m_joint_record,
         m_disjoint = m_disjoint_record,
         m_single = w_single_record,
         m_joint_noArb = m_joint_record_noArb,
         valueWind = valueWind_record,
         valueWind_noArb =valueWind_record_noArb,
         value_arbitrager = value_arbitrager_record)